In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
base_model = "meta-llama/Meta-Llama-3-8B"
new_model = "/kaggle/input/checkpoint4/kaggle/working/model-6/checkpoint-448"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

2024-07-08 00:01:39.682108: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 00:01:39.682212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 00:01:39.813076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [5]:
messages = [
    {
        "role": "user",
        "content": "How much did CTCP Eurolur reduce its investment in ACB shares at the end of the second quarter according to the 3/2023 quarterly financial report?"
    },
    
        {"role": "system", 
         "content": """Passage 1: The 10 hot stocks under the short-term and long-term trading perspective of Vietstock's PTKT are the 10 hot stocks that were analyzed in the report of the Vietstock Consulting Chamber of Commerce and Industry (BCM) to the low. BCM, CTI, DPM, FRT, GMD, MSN, TCB, TPB, VIB and VCI. These stocks were
 Passage 2: The 10 hot stocks under the Vietstock PTKT angle of view may serve the purpose of improving the short-term and long-term market sentiment. The BCG-CTCP Bamboo Capital Group's report analyzed the short-term stock prices of BCG, CTG, DPM, HSG, MSG, NLG, TCB, TPB, VJC and VIC. These stocks were
 Passage 3: The 10 hot stocks under the old Vietstock PTKT angle of view are likely to serve the main objective of improving the short-term and long-term market sentiment. AAA-CTCP Plastic Development Green Stock Market Reporting: AAA, CTG, DPM, FCN, GMD, NLG, TCB, TPB, VJC and VNM. These stocks are selected
 Passage 4: The 10 other hot stocks under the Vietstock's PTKT angle of view are likely to serve the short-term and long-term benchmark purposes of the Vietstock Consulting Chamber's report. AAA-CTCP Double-Platform Reduction of Shares Group A: AAA, CTG, DPM, FCN, GMD, NLG, TCB, TPB, VJC and VHM. These stocks
 Passage 5: The 10 hot stocks under the Vietstock Bank's PTKT viewpoint may serve the short-term and long-term benchmark purposes of the report of the Vietstock Consulting Chamber. BCG-CTCP Bamboo Capital's analyzed hot stocks include: BCG, BCM, DPM, HSG, MSN, NLG, TCB, TPB, VJC and VEA. These stocks were"""
        },
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|im_start|>user
How much did CTCP Eurolur reduce its investment in ACB shares at the end of the second quarter according to the 3/2023 quarterly financial report?<|im_end|>
<|im_start|>system
Passage 1: The 10 hot stocks under the short-term and long-term trading perspective of Vietstock's PTKT are the 10 hot stocks that were analyzed in the report of the Vietstock Consulting Chamber of Commerce and Industry (BCM) to the low. BCM, CTI, DPM, FRT, GMD, MSN, TCB, TPB, VIB and VCI. These stocks were
 Passage 2: The 10 hot stocks under the Vietstock PTKT angle of view may serve the purpose of improving the short-term and long-term market sentiment. The BCG-CTCP Bamboo Capital Group's report analyzed the short-term stock prices of BCG, CTG, DPM, HSG, MSG, NLG, TCB, TPB, VJC and VIC. These stocks were
 Passage 3: The 10 hot stocks under the old Vietstock PTKT angle of view are likely to serve the main objective of improving the short-term and long-term market sentiment. AAA-CTCP Plastic Deve

In [6]:
model.save_pretrained("model_6")
tokenizer.save_pretrained("model_6")

('model_6/tokenizer_config.json',
 'model_6/special_tokens_map.json',
 'model_6/tokenizer.json')

In [7]:
model.push_to_hub("model_6", use_temp_dir=False)
tokenizer.push_to_hub("model_6", use_temp_dir=False)

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/maithaoly/model_6/commit/8d97a1fdf9b0c5bc9f42d8c9a362ccc626814a5c', commit_message='Upload tokenizer', commit_description='', oid='8d97a1fdf9b0c5bc9f42d8c9a362ccc626814a5c', pr_url=None, pr_revision=None, pr_num=None)